In [1]:
import numpy as np
import pandas as pd

In [2]:
from my_packages.layers import Dense
from my_packages.policy import Policy
from my_packages.environment import StockEnv
from my_packages.agent import Agent

In [3]:
%load_ext autoreload
%autoreload 2

# 1. Handling Data

In [4]:
api_query = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=1min&month=2025-02&outputsize=full&datatype=csv&apikey=6QP1RCVBQ5TCGE65"

In [5]:
df = pd.read_csv("data/IBM_feb_25.csv")
df.head()

,timestamp,open,high,low,close,volume
0,2025-02-28 19:59:00,252.5,252.5,252.50,252.50,1
1,2025-02-28 19:58:00,252.5,252.5,252.03,252.03,45
2,2025-02-28 19:53:00,252.5,252.5,252.50,252.50,116
3,2025-02-28 19:51:00,252.5,252.5,252.50,252.50,15
4,2025-02-28 19:49:00,252.5,252.5,252.50,252.50,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12388 entries, 0 to 12387
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  12388 non-null  object 
 1   open       12388 non-null  float64
 2   high       12388 non-null  float64
 3   low        12388 non-null  float64
 4   close      12388 non-null  float64
 5   volume     12388 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 580.8+ KB


In [7]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [8]:
# DATA ENGINEER WITH THE DATA I'M SKIPPING THE PART AS OF NOW 
# BE CAREFULL TO CHANGE THE INDEX OF RESPECTIVE DATA IN THE environment.py

In [9]:
df = df.iloc[:,1:]

In [36]:
X = df.to_numpy(dtype = float)

In [11]:
X.shape

(12388, 5)

# 2. Building model (Policy)

In [196]:
mean_x = np.mean(X, axis = 0)
std_x = np.std(X , axis = 0)
X = (X - mean_x) / std_x
X[:5,:]

array([[-1.08844812, -1.11586902, -1.06650367, -1.09453579, -0.12039455],
       [-1.08844812, -1.11586902, -1.16510967, -1.19368412, -0.11997423],
       [-1.08844812, -1.11586902, -1.06650367, -1.09453579, -0.11929598],
       [-1.08844812, -1.11586902, -1.06650367, -1.09453579, -0.12026081],
       [-1.08844812, -1.11586902, -1.06650367, -1.09453579, -0.12036589]])

In [197]:
env = StockEnv()
env.set_env_data(X)

In [198]:
policy = Policy([Dense(units=10, activation='relu'),
                 Dense(units=3, activation='linear')],
                data_cols = 5,#REMEMBER TO CHANGE THIS AFTER FEATURE ENGINEERING 
                loss = 'mse')

In [199]:
policy.predict(X[8123])

array([[-1.13989461e-04,  5.78084470e-04,  2.09671232e-05]])

In [193]:
env.num_shares

0

In [200]:
policy.episode_train(env, batch_size = 10)

In [209]:
policy.predict(X[1234])

array([[-50.71814033, -35.31525902, -35.47974627]])

Now the policy is ready and can be deserialized and stored and reused when required

# 3. Agent using policy